In [ ]:
import numpy as np
import pandas as pd
import cv2

driving_log_filename = './data/driving_log.csv'

df = pd.read_csv(driving_log_filename)
df.head()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt

def imshow(cv2img):
    plt.imshow(cv2.cvtColor(cv2img, cv2.COLOR_BGR2RGB))
    
first_img = cv2.imread('./data/' + df['center'][0])
imshow(first_img)
plt.savefig('./img/src.png', bbox_inches='tight')
plt.title('Original image')

In [ ]:
plt.hist(df['steering'],bins=20)
plt.xlabel('steering angle')
plt.ylabel('number of obs')
plt.savefig('./img/hist.png', bbox_inches='tight')
plt.title('Histogram of steering angle')

In [ ]:
print('Image size:',first_img.shape)

In [ ]:
height = first_img.shape[0]
crop_top = height//3
crop_bottom = 4*height//5
target_size = (200,66)

def jitter_brightness(img):
    out = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    random_bright = .25 + np.random.uniform()
    out[:,:,2] = out[:,:,2]*random_bright
    out = cv2.cvtColor(out,cv2.COLOR_HSV2RGB)
    return out

def jitter_translation(img, steer, dist):
    dx = dist*(np.random.uniform()-0.5)
    dy = 40*(np.random.uniform()-0.5)
    M = np.float32([[1,0,dx],[0,1,dy]])
    rows, cols = img.shape[:2]
    out = cv2.warpAffine(img, M, (cols,rows))
    steer += dx/dist*2*.2
    return out, steer

def preprocess(img, yuv=True):
    img = cv2.resize(img[crop_top:crop_bottom,:], target_size, interpolation=cv2.INTER_AREA)
    return cv2.cvtColor(img, cv2.COLOR_BGR2YUV) if yuv else img

def read_and_jitter(row):
    cameras = ['center', 'left', 'right']
    angle_offset = [0.0, 0.25, -0.25]
    i = np.random.randint(3)
    img_filename = './data/' + row[cameras[i]][0].strip()
    img = cv2.imread(img_filename)
    steer = row['steering'][0] + angle_offset [i]
    
    img = jitter_brightness(img)
    img, steer = jitter_translation(img, steer, 100)
    
    if np.random.randint(2):
        img = cv2.flip(img, 1)
        steer = -steer
    return img, steer

post_processed = preprocess(first_img,False)
imshow(post_processed)
plt.savefig('./img/cropped.png', bbox_inches='tight')
plt.title('Cropped image')

In [ ]:
n_cols = 3
n_rows = 4
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(n_cols*4,n_rows))
gs1 = gridspec.GridSpec(n_rows,4*n_cols, top=1., bottom=0., right=1., left=0., hspace=0.,
        wspace=0.)
for i in range(n_cols*n_rows):
    ax = plt.subplot(gs1[i*4:(i+1)*4])
    img, steer = read_and_jitter(df.iloc[[0]])
    plt.imshow(preprocess(img, False))
    plt.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(10,12,'{:.3f}'.format(steer), bbox={'facecolor':'green', 'alpha':0.5, 'pad':2})
    ax.set_aspect('auto')
    
gs1.tight_layout(fig, rect=[0,0.03,1,0.97])
plt.suptitle('Jittered images and respective steering angles', size=14)


In [ ]:
def batch_generator(df, straight_pr, batch_size=32):
    batch_img = np.zeros((batch_size, *post_processed.shape))
    batch_steer = np.zeros(batch_size)
    while True:
        for i in range(batch_size):
            row = df.iloc[[np.random.randint(len(df))]].reset_index()
            x, y = read_and_jitter(row)
            if np.random.uniform() < straight_pr:
                while abs(y) < 0.1: x, y = read_and_jitter(row)
                    
            batch_img[i] = preprocess(x)
            batch_steer[i] = y
        yield batch_img, batch_steer
        

In [ ]:
def batch_generator_test(df, batch_size=32):
    batch_img = np.zeros((batch_size, *post_processed.shape))
    batch_steer = np.zeros(batch_size)
    
    start, end, n = 0, batch_size, len(df)
    while end <= n:
        j = 0
        for i in range(start, min(end, n)):
            row = df.iloc[[i]].reset_index()
            img_filename = './data/' + row['center'][0].strip()
            x = preprocess(cv2.imread(img_filename))
            batch_img[j] = x
            batch_steer[j] = row['steering'][0]
            j += 1
        start += batch_size
        end += batch_size
                                  
        yield batch_img[:j], batch_steer[:j]


In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Activation, Dense, Dropout, Lambda
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.regularizers import l2

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=post_processed.shape))
model.add(Convolution2D(24, 5, 5, subsample=(2,2), W_regularizer=l2(0.01), border_mode='valid', init='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), W_regularizer=l2(0.01), border_mode='valid', init='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), W_regularizer=l2(0.01), border_mode='valid', init='he_normal'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1,1), W_regularizer=l2(0.01), border_mode='valid', init='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1,1), W_regularizer=l2(0.01), border_mode='valid', init='he_normal'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(1024, W_regularizer=l2(0.01), init='he_normal'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(128, W_regularizer=l2(0.01), init='he_normal'))
model.add(Activation('relu'))
model.add(Dense(64, W_regularizer=l2(0.01), init='he_normal'))
model.add(Activation('relu'))
model.add(Dense(16, init='he_normal'))
model.add(Activation('relu'))
model.add(Dense(1, init='he_normal'))

In [ ]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
try:
    svg = SVG(model_to_dot(model, show_shapes=True, show_layer_names=False).create(prog='dot', format='svg'))
    with open('model.svg', 'w') as svgfile:
        svgfile.write(a.data)
except:
    svg = None
    pass
svg


In [ ]:
try:
    from keras_diagram import ascii
    print(ascii(model))
except:
    pass

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df)

In [ ]:
print('Validation set:', len(df_val))

In [ ]:
%matplotlib inline
model.compile('adam', 'mean_squared_error', [keras.metrics.mean_absolute_percentage_error])
samples_per_epoch = 20000

for epoch in range(10):
    straight_pr = 1.0/(1+epoch)
    history = model.fit_generator(batch_generator(df_train, straight_pr),
                                  samples_per_epoch,
                                  validation_data = batch_generator_test(df_val),
                                  nb_val_samples=1952,
                                  nb_epoch=1, verbose=2)


In [ ]:
with open("model.json", "w") as jsonfile:
    jsonfile.write(model.to_json())
model.save_weights("model.h5")